# DSEN Training Pipeline Step-by-Step Breakdown

This notebook provides a detailed explanation of the DSEN training pipeline code. We will cover:
- Environment setup and library imports
- Utility function definitions
- Evaluation and training functions
- Argument parsing and directory setup
- Data preparation and model initialization
- The training loop and testing phase



## Environment Setup & Imports

In [ ]:
import os
# Disable CUDA DSA for torch (if needed)
os.environ["TORCH_USE_CUDA_DSA"] = "0"

# Standard libraries
import argparse
import math
import time
import random
import warnings
from tqdm import tqdm
import logging

# Data manipulation and scientific computing
import numpy as np
import pandas as pd
import scipy

# PyTorch and related utilities
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data.distributed
from torch.nn.utils import clip_grad_norm_
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

# Additional libraries for metrics and distances
from scipy.stats import pearsonr, entropy
from scipy.spatial.distance import jensenshannon
from sklearn import metrics

# Custom modules (make sure these are in your working directory or PYTHONPATH)
from models import DSEN
from utils import Data_utility
from Optim import Optim


## Utility Function Definitions

In [ ]:
def make_dir(path, dic_name):
    """
    Create a directory if it does not exist.
    """
    path = os.path.join(path, dic_name)
    if os.path.exists(path):
        print("----Dic existed----")
    else:
        os.mkdir(path)
        print("----Dic created successfully----")
    return path

def find_xy_flow(flow_df, TCMA_cbg):
    """
    Assign latitude and longitude information to the flow dataframe based on TCMA_cbg.
    """
    TCMA_cbg_center = TCMA_cbg.copy()
    flow_df['start_lat'] = None
    flow_df['start_lon'] = None
    flow_df['end_lat'] = None
    flow_df['end_lon'] = None

    for index, row in tqdm(TCMA_cbg_center.iterrows()):
        start_lat = row['y']
        start_lon = row['x']
        end_lat = row['y']
        end_lon = row['x']
        flow_df.loc[flow_df.o_cbg == row['CensusBlockGroup'], 'start_lat'] = start_lat
        flow_df.loc[flow_df.o_cbg == row['CensusBlockGroup'], 'start_lon'] = start_lon
        flow_df.loc[flow_df.d_cbg == row['CensusBlockGroup'], 'end_lat'] = end_lat
        flow_df.loc[flow_df.d_cbg == row['CensusBlockGroup'], 'end_lon'] = end_lon

    return flow_df

def js_divergence(p, q):
    """
    Calculate the Jensen-Shannon Divergence between two distributions.
    """
    p = np.array(p) / np.sum(p)
    q = np.array(q) / np.sum(q)
    return jensenshannon(p, q) ** 2

def common_part_of_commuters(generated_flow_flat, real_flow_flat, numerator_only=False):
    """
    Compute the Common Part of Commuters (CPC) metric.
    """
    nonzero_indices = np.nonzero(real_flow_flat)
    filtered_real_flow = real_flow_flat[nonzero_indices]
    filtered_generated_flow = generated_flow_flat[nonzero_indices]

    filtered_real_flow = np.maximum(filtered_real_flow, 0)
    numerator = 2 * np.sum(np.minimum(filtered_real_flow, filtered_generated_flow))
    denominator = np.sum(filtered_real_flow + filtered_generated_flow)
    cpc = numerator / denominator if denominator != 0 else 0.0
    return cpc

def NRMSE(generated_flow, real_flow):
    """
    Compute the Normalized Root Mean Squared Error (NRMSE) for each sample.
    """
    NRMSE_val = 0.0
    for i in range(len(generated_flow)):
        select_indice = np.where(real_flow[i] != 0)[0]
        predictions = generated_flow[i][select_indice].copy()
        actuals = real_flow[i][select_indice].copy()
        mse = np.mean((predictions - actuals) ** 2)
        rmse = np.sqrt(mse)
        norm_factor = np.max(actuals) - np.min(actuals)
        if norm_factor != 0:
            NRMSE_val += rmse / norm_factor
    return NRMSE_val / len(generated_flow)


## Evaluation and Training Functions

In [ ]:
def evaluate(data, X, Y, model, test_save_path, batch_size=64, mode="valid"):
    """
    Evaluate or test the model on given data.
    """
    model.eval()
    generated_flow = []
    real_flow = []

    for x, y in data.get_batches(X, Y, batch_size):
        x = x.clone().detach().cuda()
        y = y.clone().detach().cuda()
        out, attention_weights_snap1, attention_weights_snap2, evo_embedding = model(data, x)
        generated_flow.append(out.cpu().detach().numpy())
        real_flow.append(y.cpu().numpy())

    generated_flow = np.concatenate(generated_flow)
    real_flow = np.concatenate(real_flow)

    cpc = common_part_of_commuters(generated_flow, real_flow)
    corr, _ = pearsonr(generated_flow, real_flow)
    nrmse = np.sqrt(np.mean((generated_flow - real_flow) ** 2))
    mape = np.mean(np.abs((real_flow - generated_flow) / real_flow)) * 100
    js = js_divergence(generated_flow, real_flow)

    if mode == "test":
        o_index_list = X[:, 0]
        d_index_list = X[:, 1]
        test_results = pd.DataFrame.from_dict({
            'o': o_index_list, 
            'd': d_index_list, 
            'generated_flow': generated_flow, 
            'Actual_flow': real_flow
        })
        test_results.to_csv(test_save_path + '12345/results/generated_flow_test_set.csv', index=False)

    return cpc, corr, nrmse, mape, js

def train(data, X, Y, model, criterion_mse, lambda_reg, optim, epoch, batch_size):
    """
    Train the model for one epoch.
    """
    model.train()
    total_loss = 0.0
    generated_flow = []
    real_flow = []
    evo_embedding = []

    last_attention_weights_snap1 = None
    last_attention_weights_snap2 = None
    last_edge_index_snap1 = None
    last_edge_index_snap2 = None

    for x, y in data.get_batches(X, Y, batch_size):
        model.zero_grad()
        x = x.clone().detach().cuda()
        y = y.clone().detach().cuda()
        out, attention_weights_snap1, attention_weights_snap2, Batch_evo_embedding = model(data, x)

        edge_index_snap1, attention_weights_snap1 = attention_weights_snap1
        edge_index_snap2, attention_weights_snap2 = attention_weights_snap2

        last_attention_weights_snap1 = attention_weights_snap1.cpu().detach().numpy()
        last_attention_weights_snap2 = attention_weights_snap2.cpu().detach().numpy()
        last_edge_index_snap1 = edge_index_snap1.cpu().detach().numpy()
        last_edge_index_snap2 = edge_index_snap2.cpu().detach().numpy()
        Batch_evo_embedding = Batch_evo_embedding.cpu().detach().numpy()

        batch_loss = criterion_mse(out, y)
        l2_norm = sum(p.pow(2).sum() for p in model.parameters())
        batch_loss += lambda_reg * l2_norm

        generated_flow.append(out.cpu().detach().numpy())
        real_flow.append(y.cpu().numpy())
        evo_embedding.append(Batch_evo_embedding)

        batch_loss.backward()
        grad_norm = optim.step()
        total_loss += batch_loss.cpu().data.numpy()

    generated_flow = np.concatenate(generated_flow)
    real_flow = np.concatenate(real_flow)
    evo_embedding = np.concatenate(evo_embedding)

    print("Generated Flow:", generated_flow)
    print("Real Flow:", real_flow)

    cpc = common_part_of_commuters(generated_flow, real_flow)
    corr, _ = pearsonr(generated_flow, real_flow)
    nrmse = np.sqrt(np.mean((generated_flow - real_flow) ** 2))
    mape = np.mean(np.abs((real_flow - generated_flow) / real_flow)) * 100
    js = js_divergence(generated_flow, real_flow)

    return (total_loss, cpc, corr, nrmse, mape, js, 
            last_attention_weights_snap1, last_attention_weights_snap2, 
            last_edge_index_snap1, last_edge_index_snap2, evo_embedding)


## Argument Parsing and Directory Setup

In [ ]:
# Argument parsing for model configuration and hyperparameters
parser = argparse.ArgumentParser(description='DSEN on Minneapolis')
parser.add_argument('--model', type=str, default='DSEN', help='The model of DSEN')
parser.add_argument('--epochs', type=int, default=1500, help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='batch size')
parser.add_argument('--seed', type=int, default=12345, help='random seed')
parser.add_argument('--gpu', type=int, default=0, help="The Index of GPU where we want to run the code")
parser.add_argument('--cuda', type=str, default=True)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--clip', type=float, default=10., help='gradient clipping')
parser.add_argument('--optim', type=str, default='adam')
parser.add_argument('--test_model_name', type=str, default='full_DSEN', help='the name of tested model')
args = parser.parse_args()

# Set up directories for results, logs, and model weights
path = "/"
path_result = make_dir(path, str(args.seed))
path_log = make_dir(path_result, args.test_model_name + "log")
path_model = make_dir(path_result, "pkl")
path_att_weight = make_dir(path_result, "att_weight")
path_evo_feature = make_dir(path_result, "evo_feature")
path_result = make_dir(path_result, "results")

# Configure logging
logging.basicConfig(filename=path_result + '_process.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

print("----Splitting the training and testing data----")


## Data Preparation and GPU Settings

In [ ]:
# Prepare data using a custom Data_utility class
Data = Data_utility('42')  # '42' acts as a seed for splitting data

# Configure GPU settings based on the arguments
args.cuda = args.gpu is not None
if args.cuda:
    torch.cuda.set_device(args.gpu)

# Set random seed for reproducibility
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: CUDA device is available, consider using --cuda")
    else:
        torch.cuda.manual_seed(args.seed)

# Optionally print full tensor information
torch.set_printoptions(profile='full')


## Model Initialization

In [ ]:
print("----Building models----")
# Initialize the DSEN model with specified parameters
model = eval(args.model).Model(
    node_num_features=22, 
    flow_num_features=1, 
    head=1, 
    loc_embedding_dim=24, 
    evo_emb_dim=64,
    evo_emb_hidden_dim=128, 
    flow_generator_hidden_dim=128, 
    dropout_p=0.3
)

if args.cuda:
    model.cuda()

nParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('----Number of trainable parameters: %d ----' % nParams)

# Define the mean-squared error loss function
criterion_mse = nn.MSELoss()

# Initialize the custom optimizer
optim = Optim(model.parameters(), args.optim, args.lr, args.clip)


## Training Loop

In [ ]:
best_valid_cpc = 0.0
lambda_reg = 0.0  # Regularization strength

try:
    logging.info('----Training begins----')
    last_update = 1 
    for epoch in range(1, args.epochs + 1):
        epoch_start_time = time.time()
        
        # Training for one epoch
        train_loss, train_cpc, train_corr, train_nrmse, train_mape, train_js, \
        att_w_snap1, att_w_snap2, edge_id_snap1, edge_id_snap2, evo_embed_train = train(
            Data, 
            Data.train_flow[:, [0, 1]], 
            Data.train_flow[:, 2], 
            model,
            criterion_mse,
            lambda_reg,
            optim, 
            epoch, 
            args.batch_size
        )
        
        # Validation step
        valid_cpc, valid_corr, valid_nrmse, valid_mape, valid_js = evaluate(
            Data, 
            Data.valid_flow[:, [0, 1]], 
            Data.valid_flow[:, 2], 
            model, 
            path,
            args.batch_size, 
            "valid"
        )
        
        logging.info(
            '| Epoch {:3d} | Time: {:5.2f}s | Train Loss: {:5.4f} | '
            'Train CPC: {:5.4f} | Train Corr: {:5.4f} | Train NRMSE: {:5.4f} | '
            'Train MAPE: {:5.4f} | Train JS: {:5.4f} | Valid CPC: {:5.4f} | '
            'Valid Corr: {:5.4f} | Valid NRMSE: {:5.4f} | Valid MAPE: {:5.4f} | '
            'Valid JS: {:5.4f}'.format(
                epoch, (time.time() - epoch_start_time), train_loss, train_cpc, train_corr,
                train_nrmse, train_mape, train_js, valid_cpc, valid_corr, valid_nrmse, valid_mape, valid_js
            )
        )

        # Save model if performance improves
        if best_valid_cpc < valid_cpc and valid_cpc < 1:
            logging.info("----Epoch: {}, saving model----".format(epoch))
            with open(os.path.join(path_model, args.test_model_name + "model.pkl"), 'wb') as f:
                torch.save(model, f)
            with open(os.path.join(path_log, args.test_model_name + "log.txt"), "a") as file:
                file.write("Epoch: {}, model saved.\r\n".format(epoch))
            best_valid_cpc = valid_cpc
            last_update = epoch

        # Early stopping condition: if no improvement in 200 epochs
        if epoch - last_update == 200:
            with open(os.path.join(path_model, args.test_model_name + "model_last_epoch.pkl"), 'wb') as f:
                torch.save(model, f)
            break

except KeyboardInterrupt:
    logging.info('-' * 90)
    logging.info('----Exiting training early----')
    if not os.path.isfile(os.path.join(path_model, args.test_model_name + "model_early_stop.pkl")):
        with open(os.path.join(path_model, args.test_model_name + "model_early_stop.pkl"), 'wb') as f:
            torch.save(model, f)


## Testing Phase

In [ ]:
logging.info("----Testing begins----")
# Try loading the best saved model
if os.path.isfile(os.path.join(path_model, args.test_model_name + "model.pkl")):
    with open(os.path.join(path_model, args.test_model_name + "model.pkl"), 'rb') as f:
        model = torch.load(f)
    test_cpc, test_corr, test_nrmse, test_mape, test_js = evaluate(
        Data, 
        Data.test_flow[:, [0, 1]], 
        Data.test_flow[:, 2], 
        model, 
        path,
        args.batch_size, 
        "test"
    )
    logging.info(args)
elif os.path.isfile(os.path.join(path_model, args.test_model_name + "model_last_epoch.pkl")):
    with open(os.path.join(path_model, args.test_model_name + "model_last_epoch.pkl"), 'rb') as f:
        model = torch.load(f)
    test_cpc, test_corr, test_nrmse, test_mape, test_js = evaluate(
        Data, 
        Data.test_flow[:, [0, 1]], 
        Data.test_flow[:, 2], 
        model, 
        path,
        args.batch_size, 
        "test"
    )
    logging.info(args)
else:
    with open(os.path.join(path_model, args.test_model_name + "model_early_stop.pkl"), 'rb') as f:
        model = torch.load(f)
    test_cpc, test_corr, test_nrmse, test_mape, test_js = evaluate(
        Data, 
        Data.test_flow[:, [0, 1]], 
        Data.test_flow[:, 2], 
        model, 
        path,
        args.batch_size, 
        "test"
    )
    logging.info(args)

logging.info("Test Metrics -> CPC: {:5.4f} | Corr: {:5.4f} | NRMSE: {:5.4f} | MAPE: {:5.4f} | JS: {:5.4f}".format(
    test_cpc, test_corr, test_nrmse, test_mape, test_js
))


## Running `main.py` Directly

If you prefer to run the entire pipeline as a standalone script (instead of stepping through this notebook), you can use the `main.py` file directly from the command line.

**Usage Example:**

```
python main.py```